In [ ]:
import random
import numpy as np

class BaseRecommendationSystem:
    def __init__(self, img_dict, feat_dict):
        self.img_dict = img_dict
        self.feat_dict = feat_dict
        self.prob_dict = self.get_new_dict(img_dict)
    
    def get_new_dict(self, img_dict):
        prob_dict = {}
        for img_path, tup in img_dict.items():
            s = 0  # sum
            for i in tup:
                s += self.feat_dict[i]
            prob_dict[img_path] = s / len(tup)
        return prob_dict

    def get_next_image_path(self):
        raise NotImplementedError("This method should be implemented by subclasses.")
    
    def update_probabilities(self, img_path, feedback):
        raise NotImplementedError("This method should be implemented by subclasses.")

class HeuristicRecommendationSystem(BaseRecommendationSystem):
    def __init__(self, img_dict, feat_dict):
        super().__init__(img_dict, feat_dict)

    def get_next_image_path(self):
        keys = list(self.prob_dict.keys())
        vals = list(self.prob_dict.values())
        l = range(len(vals))
        i = random.choices(l, weights=vals, k=1)[0]
        return keys[i]

    def update_probabilities(self, img_path, feedback):
        attribute_list = self.img_dict[img_path]
        if feedback.lower() == 'upvote':
            for a in attribute_list:
                self.feat_dict[a] += 2 * self.feat_dict[a]
        else:
            for a in attribute_list:
                self.feat_dict[a] -= 2/3 * self.feat_dict[a]
        self.prob_dict = self.get_new_dict(self.img_dict)

class EpsilonGreedyRecommendationSystem(BaseRecommendationSystem):
    def __init__(self, img_dict, feat_dict, epsilon=0.1):
        super().__init__(img_dict, feat_dict)
        self.epsilon = epsilon
        self.seen_images = set()

    def calculate_image_score(self, img_path):
        attribute_list = self.img_dict[img_path]
        scores = [self.feat_dict[a] for a in attribute_list]
        return np.mean(scores)

    def get_next_image_path(self):
        keys = list(self.prob_dict.keys())
        if random.random() < self.epsilon:  # Exploration
            choices = [k for k in keys if k not in self.seen_images]
            if not choices:  # If all images have been seen
                choices = keys
            selected_image = random.choice(choices)
        else:  # Exploitation
            best_image = None
            best_score = -float('inf')
            for img_path in keys:
                if img_path not in self.seen_images:
                    image_score = self.calculate_image_score(img_path)
                    if image_score > best_score:
                        best_score = image_score
                        best_image = img_path
            selected_image = best_image if best_image else random.choice(keys)
        
        self.seen_images.add(selected_image)
        return selected_image

    def update_probabilities(self, img_path, feedback):
        attribute_list = self.img_dict[img_path]
        if feedback.lower() == 'upvote':
            reward = 2
        else:
            reward = -2/3
        
        for a in attribute_list:
            self.feat_dict[a] += reward * self.feat_dict[a]
        
        self.prob_dict = self.get_new_dict(self.img_dict)


In [ ]:
#This replaces this

# Choose the algorithm
RECOMMENDATION_SYSTEM_SWITCH = 2  # 1 for Heuristic, 2 for Epsilon-Greedy

if RECOMMENDATION_SYSTEM_SWITCH == 1:
    recommender = HeuristicRecommendationSystem(img_dict, feat_dict)
elif RECOMMENDATION_SYSTEM_SWITCH == 2:
    recommender = EpsilonGreedyRecommendationSystem(img_dict, feat_dict, epsilon=0.1)

# Use the recommendation system
#next_image = recommender.get_next_image_path()
#recommender.update_probabilities(next_image, 'upvote')

is_satisfied = False

prob_dict = {}
img_p = ''


while is_satisfied == False:
  prob_dict = get_new_dict(img_dict)
  img_p = recommender.get_next_image_path()

  img = mpimg.imread(img_p)
  # fig, ax = plt.subplots()
  # ax.imshow(img)
  # ax.axis('off')
  plt.imshow(img)
    # Create a figure and axis
  plt.axis('off')

    # Display the image

  plt.show(block=False)
  plt.pause(1)
  ans = input("Enter upvote/downvote/satisfied  ->  ")
  if (ans == 'satisfied'):
    print('done')
    break
  else:
    recommender.update_probabilities(img_p, 'upvote')
